<a href="https://colab.research.google.com/github/Glifoyle/test-repo/blob/master/CVBDLU4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

--------------------------------------------------
INTRODUCTION TO COMPUTER VISION AND DEEP LEARNING
----------------------------------


# UNIT 4, HANDLING CUSTOM DATA IN PYTORCH

In this unit we continue our journey through into the world of computer vision and deep learning. Both of these are huge research areas with a lot of resources to be found online, some of them are pretty good, but some others will probably not fit your needs. We will focus on the main concepts that you need to process drone-acquired images and provide some references that may serve as a starting point if you want to dig deeper. Alternatively, if you have questions you can contact me via the habitual channels or email me at <yagodiezdonoso@gmail.com> if you have come upon these notebooks by not-so-habitual channels, everybody is welcome (just remember to introduce yourseld if we have never met. Let's get cracking then...

<span style='color:Blue'> For this unit I am going to assume that you have already completed Unit 3 of this course and, thus, you have a folder called "patches" in the same folder as this jupyter notebook that contains lots of well... patches named things like "nanakamadopatch351.jpg" an picturing lots of subalpine vegetation bushes. If you do not know what I am talking about, I suggest that you go back to the unit 3 notebook and run that one first. Alternatively (or if you are working in colab), you can just run the following cell.   </span>

In [ ]:
# Run only if you are in colab or if you have not run notebook CV3 in a while:

# first, download the data
import os
if not os.path.exists('Zao_site5'):
    !wget https://www.dropbox.com/s/isghadlhjln1qy7/Zao_site5.zip?dl=0
    !mv Zao_site5.zip?dl=0 Zao_site5.zip
    !unzip Zao_site5.zip
    !rm Zao_site5.zip

# See Unit 4 for details about what is going on here!

import cv2
import numpy as np
import os

#Read color image
def read_Color_Image(path):
    retVal = cv2.imread(path,cv2.IMREAD_COLOR)
    if retVal is None: raise Exception("Reading Color image, something went wrong with the file name "+str(path))
    return retVal

#Read binary image
def read_Binary_Mask(path):
    retVal = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    if retVal is None: raise Exception("Reading GRAYSCALE image, something went wrong with the file name "+str(path))
    retVal[retVal<=50] = 0
    retVal[retVal>50] = 255
    return retVal


def list_from_mask(mask):
    # Compute connected components
    _, _, _, centroids = cv2.connectedComponentsWithStats(255-mask)

    return centroids[1:]


def get_Square(w_size, p, img):
    return img[int(p[0])-w_size//2:int(p[0])+w_size//2, int(p[1])-w_size//2:int(p[1])+w_size//2]

def create_Patch(mosaic, center ,patchSize,convert = False):
    patchimage = get_Square(patchSize, (center[1],center[0]), mosaic)
    return cv2.cvtColor(patchimage, cv2.COLOR_BGR2RGB) if convert else patchimage

# Cut a patch around each top
def patches_from_tops(mosaic,tops,size, convert = False):
    return [create_Patch(mosaic, c ,size,convert) for c in tops]

def buildLabelImage(gtList):
    # First, build an image of the right size but with only zero values
    labelImage = np.zeros(gtList[0].shape, dtype=np.uint8)
    for i,im in enumerate(gtList):
        # The code of each class will be its position on the gtList + 1
        labelImage[im == 0] = i+1
    return labelImage


def patches_by_type(gt_names,labels,centers,patches):
    pBT = [ (t,[]) for t in gt_names ]
    for c,patch in zip(centers,patches):
        # check in the label image the code of this patch
        code = labels[int(c[1])][int(c[0])]
        if code != 0: pBT[code-1][1].append(patch)
    return pBT


def store_patches_by_type(path,pBT):
    for name,patch_list in pBT:
        for i,patch in enumerate(patch_list):
            cv2.imwrite(os.path.join(path,name+"patch"+str(i)+".jpg"),patch)

# Read Mosaic and Tops image. The "Zao_site5" folder should be in the same folder as this notebook.
pathMosaic = os.path.join("Zao_site5","site5_mosaic.jpg")
mosaic = read_Color_Image(pathMosaic)
pathTops = os.path.join("Zao_site5","site5_mosaic.jpg")
tops = read_Binary_Mask("Zao_site5/biggerTops.jpg")

# Translate from BGR to RGB (this is only to show the images in real color)
mosaic = mosaic[:,:,::-1]

# Let us find the list of centroids in our binary image
cs = list_from_mask(tops)

patches = patches_from_tops(mosaic,cs,100,True)

gt_names = ["kyaraboku","nanakamado","inutsuge","matsu","sakura","kaede"]

path_GT = os.path.join("Zao_site5","GT")
gt = [ read_Binary_Mask(os.path.join(path_GT,"site5_"+name+"_mask.png") ) for name in gt_names ]

labels = buildLabelImage(gt)

pBT = patches_by_type(gt_names,labels,cs,patches)

os.makedirs("patches", exist_ok=True)
path = os.path.normpath("patches")

store_patches_by_type(path,pBT)



--2023-06-08 05:31:41--  https://www.dropbox.com/s/isghadlhjln1qy7/Zao_site5.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/isghadlhjln1qy7/Zao_site5.zip [following]
--2023-06-08 05:31:41--  https://www.dropbox.com/s/raw/isghadlhjln1qy7/Zao_site5.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbf8775e29776be42924aeb66e6.dl.dropboxusercontent.com/cd/0/inline/B9kGW9SN8TJIJR4VBRGtluT4qaBIoPxeYN4N18KuP_n0ZcHf2qqvUdG3H7DyTnaXGALOEpLXz5tki0I2q7xTqWFFbyfkWLsQr8SaslZZkXxGVN4nZso1DFPkidmox49HJT52d73uWBUtJ4VCLJ8cARH6go-ZwuJ7gyrHdY0NfbliBw/file# [following]
--2023-06-08 05:31:42--  https://ucbf8775e29776be42924aeb66e6.dl.dropboxusercontent.com/cd/0/inline/B9kGW9SN8TJIJR4VBRGtluT4qaBIoPxeYN4N18KuP_n0ZcHf2qqvUdG3H7DyTnaXGALOEpLX

In the previous unit we turned a forest mosaic and a bunch of binary mask annotations made by the forestry experts in our team into a set of images with names that indicated the class they belonged to. We stored them all in a folder named "patches" that we have kept conveniently in the same folder as this jupyter notebook.

**In this unit we are going to take the images in that folder and turn them into a Pytorch Dataset.** This will allow us to then feed them to a deep learning model so we can train it to distinguish between our species of subalpine bushes from Zao mountain. So speaking about...

## Pytorch

[Pytorch](https://pytorch.org/) is an extremely powerful machine learning and deep learning library containing lots of useful algorithms. Pytorch also contains [Deep Learning models](https://pytorch.org/vision/stable/models.html), [convenient datasets](https://pytorch.org/vision/stable/datasets.html) that you can use to practice and lots of useful classes to proces data. Pytorch also contains lots of [tutorials](https://pytorch.org/tutorials/) that you should definetly have a look at if you want to gain a bit of a deeper (pun intended) understanding.

The way Pytoch deals with data is to use [datasets and dataloaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html). In this unit we will be dealing mostly with the first class, the dataset class, but you should be able to use the second "out of the box" after following this tutorial.

We are going to focus on creating our own "custom" Pytorch dataset, if you want to know how to use existing datasets like imagenet, MINIST or even the [Kuzushiji-MNIST](https://github.com/rois-codh/kmnist) Kanji and hiragana dataset I recommend having a look at [this tutorial](https://blog.paperspace.com/dataloaders-abstractions-pytorch/) instead.

One final word before we really do get to it (any minute now, I promise), we chose to store all of our patches with their names indicating class into one single folder. We could, just as easilly, stored them in separate folders for every class. If we had done that, then we would be having a much shorter notebook today were we just use the [ImageFolder class](https://pytorch.org/vision/stable/generated/torchvision.datasets.ImageFolder.html).

## Exercise

Change the code in the final cells in the notebook of the previous unit so the classes of each patch are stored in separate folders. Then use the ImageFolder class to build a DataLoader.

In [ ]:
## Build a Dataloader from a class folder structure using the ImageFolder class


In order to build our Datasets and DataLoaders we are going to need a few things.

In [ ]:
# important imports
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os
import re

# This will allow the images that we create to be displayed nicely within the notebook
%matplotlib inline

Again, I hope there were no big surprises there, opencv (cv2) for image manipulation (mostly reading images today), matplotlib for showing them in this notebook, numpy for array manipulation (there will be quite a lot of that today), os for file and folder management and re because we are going to extract the class name from every file name using [regular expressions](https://realpython.com/regex-python/).

In [ ]:
#pytorch imports
from torch.utils.data.dataset import Dataset

Additionally, in this lesson we have added an import related to Pytorch. In future lessons we will add a few more.

## Custom data sets in Pytorch, the Dataset class

Pytorch organizes its data using the Dataset class and then passes it onto deep learning models using the [DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) class.

The Dataset class is an __abstract class__. This means that it contains some methods that are not implemented and it cannot be instantiated. It works basically as a template that defines a certain behavior by providing some method definitions (interfaces are also often used like this, I am not going to spend time now with the differences between these two concepts).

In order to be able to use our data sets in Pytorch we need to:

1) Define a class that is a subclass of the Dataset class and define at least 3 methods that will tell Pytorch how to handle our data.   
2) The \_\_init\_\_ method or class creator that will make all the necessary preparations to load our data. In our case, this will mean accessing the "patches" folder, reading the images and assigning the correct information regarding to what bush species they depict.  
3) The \_\_getitem\_\_ method that will receive an index and provide the appropiate piece of data. In our case this will mean outputting each of our patches in turn.  
4) The \_\_len\_\_ method that outputs the size of our data. In our case, this stands for the total number of patches.

## Building our own Dataset

Let's begin with the class definition:


In [ ]:
class CPDataset(Dataset):
  pass

As we mentioned before, we need to inherint from the Dataset class. We imported this class at the start of the notebook (you can check its full definition [here](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) ) and now we are tellyng python that we will be extending this class.

Now, let's define the init method, this is were we are going to do most of the work, so be ready!

In [ ]:
class CPDataset(Dataset):
    # Given a folder containing files stored following a certain regular expression,
    # Load all image files from the folder, put them into a list
    # At the same time, load all the labels from the folder names, put them into a list too!

    def __init__(self,dataFolder):

        # Data Structures:
        self.classesDict = {} #a dictionary to store class codes
        self.imageList = [] # a list to store our images
        self.labelList = [] # a list to store our labels

        regex = re.compile('([^/]+)patch\d+.jpg$')

        # Use this if so we can also build "empty DataSets"
        if dataFolder is not None:
            # Use os.walk to obtain all the files in our folder
            files = [ f for _,_,files in os.walk(dataFolder) for f in files]
            for file in files:
                # For the files that contain "patch" in their name
                if regex.match(file):
                    # extract the image class from the file name
                    currentClass = re.split(regex,file)[1]
                    # Read the file as an opencv image
                    currentImage = cv2.imread(os.path.join(dataFolder,file))
                    if currentImage is None: raise Exception("CPDataset Constructor, problems reading file "+file)

                    # Now be careful, pytorch needs teh pixel dimension at the start,
                    # so we have to change the way the images are stored (moveaxis)
                    # We also store the image in the image list
                    self.imageList.append(np.moveaxis(currentImage,-1,0))

                    # Finally, maintain a class dictionary, the codes of the
                    # class are assigned in the order in which we encounter them
                    # also, store the label in the label list
                    if currentClass not in self.classesDict: self.classesDict[currentClass] = len(self.classesDict)
                    self.labelList.append(self.classesDict[currentClass])

Basically in the cell above we have:

1) Traversed all the files in a given folder, checking for every one of them if their name contains "patch".  
2) For those that do contain the patch word, we have extracted the word before "patch" that corresponds to the image type, in our case, to the type of bush that the image represents.  
3) We have read each image with opencv and stored it in a list (self.imageList).  
4) At the same time,  we have also stored a code corresponding to the class each image represents in another list. self.labelList. Thus, the name in the (say) third position in self.imageList contains a type of bush in dicates by the code in the third position of self.labelList.   
5) We have stored also the class name that corresponds to every code in a dictionary self.classesDict.  
6) All of this is in an \_\_init\_\_ so we have implemented a creator class that will be called every time an object of our class is initialised.  

A few technical things about the code:

1) We have allowed for the dataFolder variable to be empty. We will use this later to build empty Datasets that we will use to break one datasets into two parts, one for training our model and another for validating it.  
2) We have used regular expressions to find the images that contain our patches and to extract the bush type stored in each image. If you want to name files for an application using different naming conventions, you just need to modify the regular expression.  
3) We have had to shift the information around in each image using something called np.moveaxis. Basically opencv images have sizes like height,widht,3 if they are RGB images. We say that these are 3-channel images because every pixel contains three values. Pytorch expect the size to be stored with the channels first: 3,height,widht. You do not need to worry about this too much right now, but if you ever get an error like "Source image must have 1, 3 or 4 channels" this may be the problem. For more details see: [Pytorch vs opencv image formats](https://discuss.pytorch.org/t/plot-torch-tensor-using-opencv/20059/4).

Now let's add the \_\_getitem\_\_ and \_\_len\_\_ methods:

In [ ]:
class CPDataset(Dataset):
    # Given a folder containing files stored following a certain regular expression,
    # Load all image files from the folder, put them into a list
    # At the same time, load all the labels from the folder names, put them into a list too!

    def __init__(self,dataFolder):

        # Data Structures:
        self.classesDict = {} #a dictionary to store class codes
        self.imageList = [] # a list to store our images
        self.labelList = [] # a list to store our labels

        regex = re.compile('([^/]+)patch\d+.jpg$')

        # Use this if so we can also build "empty DataSets"
        if dataFolder is not None:
            # Use os.walk to obtain all the files in our folder
            files = [ f for _,_,files in os.walk(dataFolder) for f in files]
            for file in files:
                # For the files that contain "patch" in their name
                if regex.match(file):
                    # extract the image class from the file name
                    currentClass = re.split(regex,file)[1]
                    # Read the file as an opencv image
                    currentImage = cv2.imread(os.path.join(dataFolder,file))
                    if currentImage is None: raise Exception("CPDataset Constructor, problems reading file "+file)

                    # Now be careful, pytorch needs teh pixel dimension at the start,
                    # so we have to change the way the images are stored (moveaxis)
                    # We also store the image in the image list
                    self.imageList.append(np.moveaxis(currentImage,-1,0))

                    # Finally, maintain a class dictionary, the codes of the
                    # class are assigned in the order in which we encounter them
                    # also, store the label in the label list
                    if currentClass not in self.classesDict: self.classesDict[currentClass] = len(self.classesDict)
                    self.labelList.append(self.classesDict[currentClass])

    def __getitem__(self, index):

            inputs = self.imageList[index].astype(np.float32)

            # the target is a list of probabilities of belonging to each class
            target = np.zeros(self.numClasses())
            target[self.labelList[index]]=1

            return inputs, target

    def __len__(self):
        return len(self.imageList)

    def numClasses(self):return len(np.unique(list(self.classesDict.keys())))

As we had done most of the work before, now things are relatively simple.

- The \_\_len\_\_ method only needs to return the size of our data. In our case this is the same as the length of our list of images.

- The \_\_getitem\_\_ method needs to return two things, the data that we want to pass to a deep learning model and the "answer" to the problem that we want to solve, whatever we want the deep learning model to ... well, LEARN. In this case, we make the decision to model this second piece of information as a list of probabilities. Consequently, what we have done is define a vector with as many positions as possible bush classes and assigned a 1 to the correct position and zeros to all the other. In this way we encode that the correct response to the problem of what class this particular image belongs to is to give proabbility 1 to the correct class and probability 0 to all the others.

Notice that we have subtly changed our problems from "deciding to what bush species each image belongs to" to "assigining probabilities of belonging to each class". Notice also that we have had a casting step to float32. This happens for technical

With this type of information we will be able, in the following sessions, to train deep learning models.

Finally, we define a simple function to break a Dataset into two which will come in handy in future lessons.

In [ ]:
class CPDataset(Dataset):
    # Given a folder containing files stored following a certain regular expression,
    # Load all image files from the folder, put them into a list
    # At the same time, load all the labels from the folder names, put them into a list too!

    def __init__(self,dataFolder):

        # Data Structures:
        self.classesDict = {} #a dictionary to store class codes
        self.imageList = [] # a list to store our images
        self.labelList = [] # a list to store our labels

        regex = re.compile('([^/]+)patch\d+.jpg$')

        # Use this if so we can also build "empty DataSets"
        if dataFolder is not None:
            # Use os.walk to obtain all the files in our folder
            files = [ f for _,_,files in os.walk(dataFolder) for f in files]
            for file in files:
                # For the files that contain "patch" in their name
                if regex.match(file):
                    # extract the image class from the file name
                    currentClass = re.split(regex,file)[1]
                    # Read the file as an opencv image
                    currentImage = cv2.imread(os.path.join(dataFolder,file))
                    if currentImage is None: raise Exception("CPDataset Constructor, problems reading file "+file)

                    # Now be careful, pytorch needs teh pixel dimension at the start,
                    # so we have to change the way the images are stored (moveaxis)
                    # We also store the image in the image list
                    self.imageList.append(np.moveaxis(currentImage,-1,0))

                    # Finally, maintain a class dictionary, the codes of the
                    # class are assigned in the order in which we encounter them
                    # also, store the label in the label list
                    if currentClass not in self.classesDict: self.classesDict[currentClass] = len(self.classesDict)
                    self.labelList.append(self.classesDict[currentClass])

    def __getitem__(self, index):

            inputs = self.imageList[index].astype(np.float32)

            # the target is a list of probabilities of belonging to each class
            target = np.zeros(self.numClasses())
            target[self.labelList[index]]=1

            return inputs, target

    def __len__(self):
        return len(self.imageList)

    def numClasses(self):return len(np.unique(list(self.classesDict.keys())))


    #Create two dataset (training and validation) from an existing one
    def breakTrainValid(self,proportion):
        train=CPDataset(None)
        valid=CPDataset(None)
        train.classesDict = self.classesDict
        valid.classesDict = self.classesDict


        for i in range(int(len(self)*proportion)):
            valid.imageList.append(self.imageList[i].copy())
            valid.labelList.append(self.labelList[i])

        for i in range(int(len(self)*proportion),len(self)):
            train.imageList.append(self.imageList[i].copy())
            train.labelList.append(self.labelList[i])

        return train,valid

## Using our CPDataset class, a simple example:


Let us just use our newly minted dataset in a simple way. Remember all of those annoying axes changes that we did in \_\_init\_\_ and that matplolib and opencv do not store the images in the same format either? Let us just define a simple function to undo all those things so we can show our patches with matplolib:

In [ ]:
def to_showable_image(im):
    return np.moveaxis(im.astype(np.uint8),0,-1)[:,:,::-1]

Now we are ready, at long last, to use our Dataset class:

In [ ]:
# Create a Dataset from our "patches" folder.
da=CPDataset("patches")

# Break our dataset in two, one for training and one for testing:
tr,vd=da.breakTrainValid(0.4)

# check how many images we have in each:
print(len(tr))
print(len(vd))

# Check also how many classe we have
print(vd.numClasses())

1260
839
6


Notice that in da=CPDataset("patches") we are calling the \_\_init\_\_ method that we defined before.

In [ ]:
# Now, let's use \_\_getitem\_\_ that is called every time an object of class Dataset is indexed with []
patches_to_show = [ to_showable_image(tr[i][0]) for i in range(len(tr)) ]

Here we have used a list comprehension to repeatedly call tr\[i\] which goes to the "tr" dataset and calls the \_\_getitem\_\_ method with "index" i. in each case, we have kept the \[0\] element of the output that contains the image and ignored the corresponding label.

Now we are ready to show the patches that we read using a Pytorch dataset.

In [ ]:
# select and show a few random patches
toShow = 30

np.random.shuffle(patches_to_show)

f, axarr = plt.subplots(toShow//5,5)
plt.rcParams['figure.figsize'] = [100, 100]
for i in range(toShow//5):
    for j in range(5):
        axarr[i][j].imshow(patches_to_show[i*5+j])


## Exercises:

### Exercise 1:

Use the Dataset that we have just coded to count how many elements of each class are there in our patches folder. You can check that your response is correct by looking at the indices in the files.

In [ ]:
# Exercise 1


### Exercise 2:

Build a Dataset for a classifier of images belonging to a subject of your interest you. I am going to use an actor classifier.  

1) Download images from the internet belonging to some of your favourite actors.  Use 50 to 100 images per each actor, name them in a way that tell you what actor every image belongs to.  
2) Store all the images in the same folder.  
3) Adapt the CPDataset class to read your images and show them as we have done up here.  


In [ ]:
# Exercise 2